# Self attention

In [7]:
import torch
import torch.nn as nn

class MultiHeadedAttention(nn.Module):
    def __init__(self, d_in, d_out, context_length, dropout, num_heads, qkv_bias=False):
        super().__init__()

        assert(d_out % num_heads == 0), 'd_out should be divisible by num_heads'

        self.num_heads = num_heads
        self.d_out = d_out
        self.head_dim = d_out // num_heads
        self.W_query = nn.Linear(d_in, d_out, bias=qkv_bias)
        self.W_key = nn.Linear(d_in, d_out, bias=qkv_bias)
        self.W_value = nn.Linear(d_in, d_out, bias=qkv_bias)
        self.out_projection = nn.Linear(d_out, d_out)
        self.dropout = nn.Dropout(dropout)
        self.register_buffer (
            'mask',
            torch.triu (
                torch.ones(context_length, context_length),
                diagonal=1
            )
        )

    def forward(self, x):
        b, num_tokens, d_in = x.shape
        keys = self.W_key(x)
        queries = self.W_query(x)
        values = self.W_value(x)

        keys = keys.view(b, num_tokens, self.num_heads, self.head_dim)
        values = values.view(b, num_tokens, self.num_heads, self.head_dim)
        queries = queries.view(b, num_tokens, self.num_heads, self.head_dim)

        keys = keys.transpose(1, 2)
        queries = queries.transpose(1, 2)
        values = values.transpose(1, 2)

        attention_scores = queries @ keys.transpose(2, 3)
        mask_bool = self.mask.bool()[:num_tokens, :num_tokens]
        
        attention_scores.masked_fill_(mask_bool, -torch.inf)

        attention_weights = torch.softmax(attention_scores / keys.shape[-1]**0.5, dim=-1)
        attention_weights = self.dropout(attention_weights)

        context_vec = (attention_weights @ values).transpose(1, 2)
        context_vec = context_vec.contiguous().view(b, num_tokens, self.d_out)

        context_vec = self.out_projection(context_vec)

        return context_vec

In [8]:
inputs = torch.tensor( 
    [
        [0.43, 0.15, 0.89], # Your (x^1) 
        [0.55, 0.87, 0.66], # journey (x^2) 
        [0.57, 0.85, 0.64], # starts (x^3) 
        [0.22, 0.58, 0.33], # with (x^4) 
        [0.77, 0.25, 0.10], # one (x^5) 
        [0.05, 0.80, 0.55]  # step (x^6)
    ]
)

batch = torch.stack((inputs, inputs), dim=0)

batch_size, context_length, d_in = batch.shape
d_out = 2
mha = MultiHeadedAttention(d_in, d_out, context_length, dropout=0.0, num_heads=2)
context_vecs = mha(batch)

print(context_vecs)
print(context_vecs.shape)

tensor([[[0.1804, 0.8453],
         [0.2218, 0.8860],
         [0.2339, 0.8989],
         [0.1611, 0.8333],
         [0.1105, 0.7960],
         [0.1038, 0.7840]],

        [[0.1804, 0.8453],
         [0.2218, 0.8860],
         [0.2339, 0.8989],
         [0.1611, 0.8333],
         [0.1105, 0.7960],
         [0.1038, 0.7840]]], grad_fn=<ViewBackward0>)
torch.Size([2, 6, 2])
